# 🚢 EPA Titanic — LinearSVC V1 (Máquina de Vectores de Soporte Lineal)

En este notebook probaremos **LinearSVC** (SVM lineal con el optimizador **liblinear/LinearSVC** de scikit-learn).

## 🔎 ¿Qué es LinearSVC y en qué se diferencia?
- **LinearSVC** aprende un **clasificador lineal de máximo margen** usando la **pérdida hinge** (o squared hinge) con regularización **L2** (también soporta L1 con `dual=False`).  
- A diferencia de `SVC(kernel="linear")`, que usa el solver de SMO y puede escalar peor con muchas features, **LinearSVC** está optimizado para problemas lineales y con muchas columnas (como tras **OneHotEncoder**).  
- A diferencia de `SGDClassifier(loss="hinge")`, que entrena por **descenso de gradiente estocástico**, **LinearSVC** usa un optimizador determinista (coordinate descent/LibLinear), lo que suele dar **soluciones más estables** (menos varianza) a costa de ser algo más lento.

## 🎯 Objetivo
1. Reutilizar **Feature Engineering** y **preprocesamiento** (imputación, escalado, OneHot).  
2. Entrenar un pipeline con **LinearSVC** y validar con **5-fold estratificado**.  
3. Hacer **GridSearch** de hiperparámetros (`C`, `penalty`, `loss`, `dual`) para mejorar desempeño.  
4. Generar `submission.csv` y registrar el resultado.


In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


# 📥 Celda 2 — Carga de datos
Leemos `train.csv` y `test.csv` desde `/kaggle/input/titanic/`, verificamos dimensiones y mostramos una vista previa.


In [2]:
import pandas as pd  # manejo de DataFrames (tablas)

# Leemos los datos oficiales del entorno de competición
train_df = pd.read_csv("/kaggle/input/titanic/train.csv")  # incluye Survived
test_df  = pd.read_csv("/kaggle/input/titanic/test.csv")   # sin Survived

# Mostramos formas esperadas (sanity check)
print("Shape train:", train_df.shape)  # esperado ~ (891, 12)
print("Shape test :", test_df.shape)   # esperado ~ (418, 11)

# Vista rápida de columnas y tipos
train_df.head()  # primeras 5 filas


Shape train: (891, 12)
Shape test : (418, 11)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# 🔎 Celda 3 — Exploración inicial (EDA breve)
En esta celda:  
- Listamos las columnas del dataset de entrenamiento.  
- Revisamos valores faltantes en `train.csv` y `test.csv` (conteo y porcentaje).  
- Analizamos la distribución de la variable objetivo `Survived` para entender el balance de clases.


In [3]:
import numpy as np  # librería de utilidades numéricas

# --- Listado de columnas ---
print("Columnas en train:", train_df.columns.tolist())

# --- Faltantes en TRAIN ---
na_train = train_df.isna().sum()                          # cuenta valores nulos
na_train_pct = (na_train / len(train_df)).round(3)        # porcentaje de nulos
faltantes_train = pd.DataFrame({"nulos": na_train, "porcentaje": na_train_pct})
print("\nFaltantes en train (conteo y %):")
print(faltantes_train.sort_values("nulos", ascending=False))

# --- Faltantes en TEST ---
na_test = test_df.isna().sum()
na_test_pct = (na_test / len(test_df)).round(3)
faltantes_test = pd.DataFrame({"nulos": na_test, "porcentaje": na_test_pct})
print("\nFaltantes en test (conteo y %):")
print(faltantes_test.sort_values("nulos", ascending=False))

# --- Distribución de Survived ---
y_counts = train_df["Survived"].value_counts().sort_index()
y_ratio = train_df["Survived"].value_counts(normalize=True).round(4)

print("\nDistribución de Survived (conteo):")
print(y_counts)
print("\nDistribución de Survived (proporción):")
print(y_ratio)


Columnas en train: ['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']

Faltantes en train (conteo y %):
             nulos  porcentaje
Cabin          687       0.771
Age            177       0.199
Embarked         2       0.002
PassengerId      0       0.000
Survived         0       0.000
Pclass           0       0.000
Name             0       0.000
Sex              0       0.000
SibSp            0       0.000
Parch            0       0.000
Ticket           0       0.000
Fare             0       0.000

Faltantes en test (conteo y %):
             nulos  porcentaje
Cabin          327       0.782
Age             86       0.206
Fare             1       0.002
PassengerId      0       0.000
Pclass           0       0.000
Name             0       0.000
Sex              0       0.000
SibSp            0       0.000
Parch            0       0.000
Ticket           0       0.000
Embarked         0       0.000

Distribución de Survi

# 🧱 Celda 4 — Definición de variables base y objetivo
En esta celda:  
- Seleccionamos las columnas **numéricas** y **categóricas** originales como punto de partida.  
- Construimos la matriz de features `X` (entrenamiento) y `X_test` (predicción).  
- Definimos la variable objetivo `y` (`Survived`).  
- Guardamos `PassengerId` del test para usarlo al crear el `submission.csv`.


In [6]:
# --- Definición de columnas base ---
# Variables numéricas originales
base_num_features = ["Age", "SibSp", "Parch", "Fare", "Pclass"]

# Variables categóricas originales
base_cat_features = ["Sex", "Embarked"]

# --- Construcción de X (features de entrenamiento) ---
X = train_df[base_num_features + base_cat_features].copy()

# --- Variable objetivo ---
y = train_df["Survived"].astype(int)  # aseguramos tipo entero (0/1)

# --- Construcción de X_test (features de predicción) ---
X_test = test_df[base_num_features + base_cat_features].copy()

# --- Guardamos PassengerId (para el submission) ---
test_passenger_id = test_df["PassengerId"].copy()

# Vista previa de X
X.head()


,Age,SibSp,Parch,Fare,Pclass,Sex,Embarked
0,22.0,1,0,7.2500,3,male,S
1,38.0,1,0,71.2833,1,female,C
2,26.0,0,0,7.9250,3,female,S
3,35.0,1,0,53.1000,1,female,S
4,35.0,0,0,8.0500,3,male,S


# 🧪 Celda 5 — Ingeniería de atributos
En esta celda crearemos nuevas variables derivadas de las originales para enriquecer el modelo:

- **FamilySize** = SibSp + Parch + 1  
- **IsAlone** = 1 si FamilySize == 1, en caso contrario 0  
- **Title** = extraído de la columna Name (agrupando títulos raros en "Other")  
- **CabinKnown** = 1 si Cabin no es NaN, 0 en caso contrario  
- **FarePerPerson** = Fare dividido por FamilySize  
- **TicketGroupSize** = número de pasajeros que comparten el mismo Ticket  

Estas transformaciones se implementarán en una función y se integrarán en el pipeline con `FunctionTransformer`.


In [7]:
from sklearn.preprocessing import FunctionTransformer  # para aplicar funciones personalizadas en un Pipeline

# Definimos la función de ingeniería de atributos
def add_features(df):
    df = df.copy()

    # --- FamilySize e IsAlone ---
    df["FamilySize"] = df["SibSp"].fillna(0) + df["Parch"].fillna(0) + 1
    df["IsAlone"] = (df["FamilySize"] == 1).astype(int)

    # --- Title (extraído de Name) ---
    if "Name" in df.columns:
        titles = df["Name"].str.extract(r",\s*([^\.]+)\.")[0]
        titles = titles.replace({"Mlle": "Miss", "Ms": "Miss", "Mme": "Mrs"})
        titles = titles.where(titles.isin(["Mr","Mrs","Miss","Master"]), "Other")
        df["Title"] = titles.fillna("Other")
    else:
        df["Title"] = "Other"

    # --- CabinKnown ---
    if "Cabin" in df.columns:
        df["CabinKnown"] = (~df["Cabin"].isna()).astype(int)
    else:
        df["CabinKnown"] = 0

    # --- FarePerPerson ---
    df["FarePerPerson"] = df["Fare"].fillna(df["Fare"].median()) / df["FamilySize"].replace(0, 1)

    # --- TicketGroupSize ---
    if "Ticket" in df.columns:
        counts = df["Ticket"].map(df["Ticket"].value_counts())
        df["TicketGroupSize"] = counts.fillna(1).astype(int)
    else:
        df["TicketGroupSize"] = 1

    return df

# Creamos el transformador para usar dentro del pipeline
feature_engineering = FunctionTransformer(add_features, validate=False)

# Añadimos columnas auxiliares necesarias a X y X_test (Name, Ticket, Cabin)
for col in ["Name", "Ticket", "Cabin"]:
    if col not in X.columns and col in train_df.columns:
        X[col] = train_df[col]
    if col not in X_test.columns and col in test_df.columns:
        X_test[col] = test_df[col]

print("✅ Ingeniería de atributos lista (FunctionTransformer definido y columnas auxiliares añadidas).")


✅ Ingeniería de atributos lista (FunctionTransformer definido y columnas auxiliares añadidas).


# 🧽 Celda 6 — Preprocesamiento (imputación, escalado y One-Hot)
Vamos a preparar un `ColumnTransformer` que:
- Imputa **numéricas** con **mediana** y luego **escala** (SVM/LinearSVC es sensible a magnitudes).
- Imputa **categóricas** con **moda** y aplica **One-Hot** (incluye `Title` creado en la Celda 5).

> Nota: Solo listamos las **features finales** que entran al modelo.  
> Las columnas auxiliares (`Name`, `Ticket`, `Cabin`) no se usan directamente, solo sirven para crear features.


In [8]:
# Importamos utilidades para construir el preprocesamiento por tipo de columna
from sklearn.compose import ColumnTransformer            # aplicar distintos transformadores por columna
from sklearn.pipeline import Pipeline                    # encadenar pasos (FE -> pre -> modelo)
from sklearn.impute import SimpleImputer                 # imputación de faltantes
from sklearn.preprocessing import StandardScaler, OneHotEncoder  # escalado y one-hot

# Definimos las columnas que EXISTIRÁN DESPUÉS del Feature Engineering (Celda 5)
num_features = [                                         # variables numéricas finales
    "Age", "SibSp", "Parch", "Fare", "Pclass",           # numéricas originales
    "FamilySize", "IsAlone", "FarePerPerson",            # creadas
    "TicketGroupSize", "CabinKnown"                      # creadas (CabinKnown es binaria, la tratamos como numérica)
]
cat_features = [                                         # variables categóricas finales
    "Sex", "Embarked", "Title"                           # Title proviene de la Celda 5
]

# Pipeline para numéricas: imputar con mediana y escalar (muy importante para LinearSVC)
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),       # rellena NaN en numéricas con la mediana
    ("scaler", StandardScaler())                         # estandariza (media 0, desvío 1)
])

# Pipeline para categóricas: imputar con moda y codificar con One-Hot
categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),             # rellena NaN en categóricas con la moda
    ("onehot", OneHotEncoder(handle_unknown="ignore",                 # crea dummies; ignora categorías no vistas
                             sparse_output=False))                    # salida densa (dataset es pequeño)
])

# Componemos el ColumnTransformer, aplicando cada pipeline a sus columnas
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_features),      # aplica pipeline numérico a num_features
        ("cat", categorical_transformer, cat_features)   # aplica pipeline categórico a cat_features
    ],
    remainder="drop"                                     # descarta columnas no listadas (Name/Ticket/Cabin)
)

# Mensajes de verificación
print("✅ Preprocesamiento definido para LinearSVC.")
print("Num features:", num_features)
print("Cat features:", cat_features)


✅ Preprocesamiento definido para LinearSVC.
Num features: ['Age', 'SibSp', 'Parch', 'Fare', 'Pclass', 'FamilySize', 'IsAlone', 'FarePerPerson', 'TicketGroupSize', 'CabinKnown']
Cat features: ['Sex', 'Embarked', 'Title']


# 🤖 Celda 7 — Pipeline (FE → Preprocesamiento → LinearSVC) + Validación Cruzada
Encadenamos la **Ingeniería de Features**, el **preprocesamiento** y el modelo **LinearSVC** en un `Pipeline`.
Evaluamos con **5-fold Stratified CV** usando *accuracy* (métrica oficial de la competición).

> Notas de configuración:
> - `LinearSVC` usa la pérdida **squared_hinge** por defecto (SVM lineal).
> - Con **más muestras que features** (nuestro caso tras OneHot), suele ser más eficiente `dual=False`.
> - El modelo es sensible a la escala → ya estandarizamos en el preprocesamiento.


In [9]:
# Importamos el modelo y utilidades de evaluación
from sklearn.svm import LinearSVC                          # SVM lineal (optimización determinista)
from sklearn.model_selection import StratifiedKFold, cross_val_score  # CV estratificada y evaluación
from sklearn.pipeline import Pipeline                      # para encadenar pasos
import numpy as np                                         # utilidades numéricas

# Definimos un LinearSVC "baseline" robusto
linsvc = LinearSVC(
    penalty="l2",        # regularización L2 (estándar y estable)
    loss="squared_hinge",# pérdida por defecto en LinearSVC
    C=1.0,               # fuerza de regularización (más alto = menos regularización)
    dual=False,          # con n_samples > n_features suele ser más eficiente desactivar dual
    tol=1e-4,            # tolerancia de parada
    max_iter=5000        # iteraciones máximas (evita warnings de convergencia)
    # class_weight="balanced"  # <- opcional para compensar leve desbalance 0/1
)

# Construimos el pipeline completo: FE -> Preprocesamiento -> Modelo
pipe_linsvc = Pipeline(steps=[
    ("fe", feature_engineering),  # Celda 5: genera FamilySize, Title, etc.
    ("pre", preprocessor),        # Celda 6: imputación + escalado + OneHot
    ("linsvc", linsvc)            # clasificador LinearSVC
])

# Definimos el esquema de validación cruzada estratificada
cv = StratifiedKFold(
    n_splits=5,       # 5 folds
    shuffle=True,     # barajamos antes de partir
    random_state=42   # reproducibilidad en el split
)

# Ejecutamos cross-validation con accuracy
cv_scores = cross_val_score(
    estimator=pipe_linsvc,  # pipeline completo
    X=X,                    # features base (FE se aplica dentro del pipeline)
    y=y,                    # objetivo (Survived)
    cv=cv,                  # esquema de validación
    scoring="accuracy",     # métrica oficial de la competencia
    n_jobs=-1               # usa todos los núcleos disponibles
)

# Mostramos resultados por fold y resumen
print("Accuracy por fold:", np.round(cv_scores, 4))                 # accuracies individuales
print("Accuracy promedio (CV):", cv_scores.mean().round(4),         # promedio de CV
      "| Desv. std:", cv_scores.std().round(4))                     # dispersión entre folds


Accuracy por fold: [0.8268 0.8146 0.8258 0.8371 0.8427]
Accuracy promedio (CV): 0.8294 | Desv. std: 0.0097


# 📤 Celda 8 — Entrenamiento final y creación de `submission.csv`
Entrenamos el **pipeline completo con LinearSVC** en **todo `train`** y generamos el archivo
`submission.csv` (formato Kaggle: `PassengerId`, `Survived`, 418 filas).


In [10]:
# 1) Entrenamos el pipeline completo (FE → Preprocesamiento → LinearSVC) con TODO el train
pipe_linsvc.fit(X, y)  # ajusta usando todas las filas de entrenamiento

# 2) Obtenemos predicciones binarias (0/1) sobre el set de test
test_preds = pipe_linsvc.predict(X_test)  # LinearSVC no tiene predict_proba; trabajamos con clases
test_preds = test_preds.astype(int)       # nos aseguramos de que la salida sea entera

# 3) Construimos el DataFrame en el formato requerido por Kaggle
submission = pd.DataFrame({
    "PassengerId": test_passenger_id,  # IDs originales del conjunto de test
    "Survived": test_preds             # predicciones del modelo
})

# 4) Chequeos defensivos de formato (muy útil para evitar errores de subida)
assert submission.shape[0] == 418, "El submission debe tener exactamente 418 filas."
assert list(submission.columns) == ["PassengerId", "Survived"], "Columnas deben ser PassengerId y Survived."

# 5) Guardamos el CSV en disco (Kaggle lo detecta para la pestaña 'Submit Predictions')
submission.to_csv("submission.csv", index=False)  # guarda sin índice

# 6) Mensajes de confirmación y vista previa
print("✅ Archivo 'submission.csv' creado con", submission.shape[0], "filas.")
print(submission.head())  # primeras 5 filas para inspección rápida
print("\nConteo predicciones:", submission["Survived"].value_counts().to_dict())  # distribución 0/1


✅ Archivo 'submission.csv' creado con 418 filas.
   PassengerId  Survived
0          892         0
1          893         1
2          894         0
3          895         0
4          896         1

Conteo predicciones: {0: 253, 1: 165}


# 🏆 Celda 9 — Registro del resultado

- **Score Kaggle:** 0.77272  
- **Notebook:** EPA_TITANIC_LINSVC_V1.ipynb  
- **Modelo:** LinearSVC (máquina de vectores de soporte lineal)  
- **Hiperparámetros:**  
  - penalty = "l2"  
  - loss = "squared_hinge"  
  - C = 1.0  
  - dual = False  
  - max_iter = 5000  
- **Resultados:**  
  - CV promedio: 0.8294 (std 0.0097)  
  - Kaggle score: 0.77272 (empatado con LogReg V1, ligeramente peor que SGD V1)  
- **Notas:**  
  - El modelo fue muy estable en CV, pero en Kaggle no superó a SGD.  
  - Próximos pasos: probar ensambles de árboles (Random Forest, Gradient Boosting) para buscar un salto mayor.  
